In [39]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import RandomZoom, RandomRotation, RandomFlip, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pandas as pd


In [40]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Set seed
SEED = 338424

# Global variables
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
num_classes = 18 # Number of folders in dataset
AUTOTUNE = tf.data.AUTOTUNE

In [41]:
# Load Dataset
dataset_dir = 'dataset/hagridset'
full_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    seed=SEED,
    image_size=(IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Split into training, validation, and test sets
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Total length of the dataset
total_size = len(full_ds)

# Compute indices for the splits
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - (train_size + val_size)

# # Split the dataset
# train_ds = full_ds.take(train_size)
# val_ds = full_ds.skip(train_size).take(val_size)
# test_ds = full_ds.skip(train_size + val_size)

# Split the dataset and shuffle
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED)
test_ds = full_ds.skip(train_size + val_size).shuffle(test_size, seed=SEED)

# Count samples in each subset
def count_samples(dataset):
    sample_count = sum(1 for _ in dataset.unbatch())
    return sample_count

# Output the number of samples for each dataset
print(f'Using {count_samples(train_ds)} samples in the Training set')
print(f'Using {count_samples(val_ds)} samples in the Validation set')
print(f'Using {count_samples(test_ds)} samples in the Test set')

Found 125912 files belonging to 18 classes.
Using 88128 samples in the Training set
Using 25184 samples in the Validation set
Using 12600 samples in the Test set


In [42]:
# Get class names
class_names = full_ds.class_names
class_names

['call',
 'dislike',
 'fist',
 'four',
 'like',
 'mute',
 'ok',
 'one',
 'palm',
 'peace',
 'peace_inverted',
 'rock',
 'stop',
 'stop_inverted',
 'three',
 'three2',
 'two_up',
 'two_up_inverted']

# **Light CNN**

In [5]:
# Define the Light CNN Model from Scratch
def build_scratch_cnn_light():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))  # Reduced filter size
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))  # Smaller second layer
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))  # Smaller third layer
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))  # Reduced fully connected layer
    model.add(layers.Dropout(0.3))  # Reduced dropout

    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the lighter model
scratch_model_light = build_scratch_cnn_light()
scratch_model_light.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       1

In [6]:
# Train Light CNN Model
history_custom = scratch_model_light.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 73s 17ms/step - loss: 2.7318 - accuracy: 0.1088 - val_loss: 2.5562 - val_accuracy: 0.1630
Epoch 2/10
2754/2754 [==============================] - 69s 17ms/step - loss: 2.5284 - accuracy: 0.1635 - val_loss: 2.4230 - val_accuracy: 0.1966
Epoch 3/10
2754/2754 [==============================] - 69s 17ms/step - loss: 2.3337 - accuracy: 0.2251 - val_loss: 2.2527 - val_accuracy: 0.2769
Epoch 4/10
2754/2754 [==============================] - 69s 17ms/step - loss: 1.9908 - accuracy: 0.3259 - val_loss: 1.7009 - val_accuracy: 0.4507
Epoch 5/10
2754/2754 [==============================] - 70s 18ms/step - loss: 1.6145 - accuracy: 0.4530 - val_loss: 1.7430 - val_accuracy: 0.4587
Epoch 6/10
2754/2754 [==============================] - 70s 18ms/step - loss: 1.3367 - accuracy: 0.5484 - val_loss: 1.0862 - val_accuracy: 0.6582
Epoch 7/10
2754/2754 [==============================] - 71s 18ms/step - loss: 1.1605 - accuracy: 0.6104 - val_loss: 1.0368 -

In [7]:
# Evaluate the Light CNN Model
scratch_model_light.evaluate(test_ds)

394/394 [==============================] - 34s 5ms/step - loss: 0.8619 - accuracy: 0.7297


[0.8618804812431335, 0.7296825647354126]

In [ ]:
# Save the Deep CNN Model
scratch_model_light.save('scratch_model_deep.h5')

In [ ]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_light = load_model('scratch_model_light.h5')

In [9]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_light, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 315ms/step
Prediction for myImages/three2.png: three2
1/1 [==============================] - 0s 17ms/step
Prediction for myImages/rock.png: peace_inverted
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/ok.jpg: call
1/1 [==============================] - 0s 20ms/step
Prediction for myImages/two_up_inverted.png: two_up_inverted


# **Deep CNN**

In [14]:
# Define the Deep CNN Model from Scratch
def build_scratch_cnn():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5)) 
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the model
scratch_model_deep = build_scratch_cnn()
scratch_model_deep.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 batch_normalization_10 (Bat  (None, 128, 128, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 32)        9248      
                                                                 
 batch_normalization_11 (Bat  (None, 64, 64, 32)      

In [15]:
# Train Deep CNN Model
history_custom = scratch_model_deep.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 146s 42ms/step - loss: 2.7876 - accuracy: 0.1516 - val_loss: 2.1395 - val_accuracy: 0.3644
Epoch 2/10
2754/2754 [==============================] - 115s 33ms/step - loss: 1.7598 - accuracy: 0.5011 - val_loss: 1.4110 - val_accuracy: 0.6528
Epoch 3/10
2754/2754 [==============================] - 119s 34ms/step - loss: 1.2307 - accuracy: 0.7230 - val_loss: 0.9232 - val_accuracy: 0.8284
Epoch 4/10
2754/2754 [==============================] - 122s 35ms/step - loss: 1.0267 - accuracy: 0.7932 - val_loss: 0.8557 - val_accuracy: 0.8444
Epoch 5/10
2754/2754 [==============================] - 126s 36ms/step - loss: 0.9197 - accuracy: 0.8278 - val_loss: 0.8916 - val_accuracy: 0.8324
Epoch 6/10
2754/2754 [==============================] - 121s 35ms/step - loss: 0.8551 - accuracy: 0.8466 - val_loss: 0.7245 - val_accuracy: 0.8816
Epoch 7/10
2754/2754 [==============================] - 123s 35ms/step - loss: 0.8029 - accuracy: 0.8606 - val_loss: 0

In [16]:
# Evaluate the Deep CNN Model
scratch_model_deep.evaluate(test_ds)

394/394 [==============================] - 46s 6ms/step - loss: 0.6876 - accuracy: 0.8775


[0.6875975131988525, 0.8775396943092346]

In [24]:
# Save the Deep CNN Model
scratch_model_deep.save('scratch_model_deep.h5')

In [25]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_deep = load_model('scratch_model_deep.h5')

In [27]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_deep, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 126ms/step
Prediction for myImages/three2.png: three2
1/1 [==============================] - 0s 20ms/step
Prediction for myImages/rock.png: rock
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/ok.jpg: ok
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/two_up_inverted.png: two_up_inverted


# **CNN Data Augmentation**

In [44]:
# Data Augmentation
data_augmentation_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

In [22]:
# Define the Deep CNN Model from Scratch
def build_scratch_cnn():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(data_augmentation_layers)
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5)) 
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the model
scratch_model_da = build_scratch_cnn()
scratch_model_da.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_6 (Sequential)   (None, 64, 64, 3)         0         
                                                                 
 rescaling_3 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_16 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_16 (Bat  (None, 64, 64, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 32)       

In [23]:
# Train Deep CNN Model
history_custom = scratch_model_da.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10


2754/2754 [==============================] - 428s 133ms/step - loss: 2.9181 - accuracy: 0.1024 - val_loss: 2.4708 - val_accuracy: 0.2285
Epoch 2/10
2754/2754 [==============================] - 385s 131ms/step - loss: 2.2598 - accuracy: 0.3096 - val_loss: 1.6261 - val_accuracy: 0.5374
Epoch 3/10
2754/2754 [==============================] - 383s 131ms/step - loss: 1.7588 - accuracy: 0.5098 - val_loss: 1.3032 - val_accuracy: 0.6725
Epoch 4/10
2754/2754 [==============================] - 384s 131ms/step - loss: 1.5329 - accuracy: 0.5959 - val_loss: 1.1477 - val_accuracy: 0.7220
Epoch 5/10
2754/2754 [==============================] - 379s 130ms/step - loss: 1.4032 - accuracy: 0.6475 - val_loss: 1.1871 - val_accuracy: 0.7120
Epoch 6/10
2754/2754 [==============================] - 379s 130ms/step - loss: 1.3087 - accuracy: 0.6809 - val_loss: 1.0383 - val_accuracy: 0.7568
Epoch 7/10
2754/2754 [==============================] - 381s 131ms/step - loss: 1.2507 - accuracy: 0.7000 - val_loss: 1.038

In [24]:
# Evaluate the CNN Data Augmentation Deep Model
scratch_model_da.evaluate(test_ds)

394/394 [==============================] - 41s 6ms/step - loss: 0.9589 - accuracy: 0.7879


[0.958897054195404, 0.7878571152687073]

In [ ]:
# Save the Deep CNN Model
scratch_model_da.save('scratch_model_deep.h5')

In [ ]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_da = load_model('scratch_model_deep.h5')

In [ ]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_da, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

# **Data Augmentation without Dropout/Regularization**

In [45]:
# Define the Deep CNN Model from Scratch
def build_scratch_cnn():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(data_augmentation_layers)
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
   
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
   
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
   
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
   
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
  
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())

    model.add(layers.Flatten())
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the model
scratch_model_da2 = build_scratch_cnn()
scratch_model_da2.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_13 (Sequential)  (None, 64, 64, 3)         0         
                                                                 
 rescaling_7 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_37 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_37 (Bat  (None, 64, 64, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 32, 32, 32)      

In [46]:
# Train Deep CNN Model
history_custom = scratch_model_da2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 379s 129ms/step - loss: 1.9842 - accuracy: 0.3670 - val_loss: 1.2109 - val_accuracy: 0.6120
Epoch 2/10
2754/2754 [==============================] - 374s 128ms/step - loss: 1.0563 - accuracy: 0.6568 - val_loss: 0.8299 - val_accuracy: 0.7273
Epoch 3/10
2754/2754 [==============================] - 377s 129ms/step - loss: 0.8142 - accuracy: 0.7367 - val_loss: 0.7506 - val_accuracy: 0.7550
Epoch 4/10
2754/2754 [==============================] - 376s 129ms/step - loss: 0.7357 - accuracy: 0.7595 - val_loss: 0.9575 - val_accuracy: 0.6875
Epoch 5/10
2754/2754 [==============================] - 375s 128ms/step - loss: 0.6553 - accuracy: 0.7878 - val_loss: 0.6981 - val_accuracy: 0.7694
Epoch 6/10
2754/2754 [==============================] - 378s 129ms/step - loss: 0.6018 - accuracy: 0.8053 - val_loss: 0.5978 - val_accuracy: 0.8110
Epoch 7/10
2754/2754 [==============================] - 375s 128ms/step - loss: 0.5512 - accuracy: 0.8204 - val_

In [47]:
# Evaluate the CNN Data Augmentation Deep Model
scratch_model_da2.evaluate(test_ds)

394/394 [==============================] - 36s 10ms/step - loss: 0.5184 - accuracy: 0.8387


[0.5184229016304016, 0.8386508226394653]

In [48]:
# Save the Deep CNN Model
scratch_model_da2.save('scratch_model_da2.h5')

In [ ]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_da2 = load_model('scratch_model_da2.h5')

In [52]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_da2, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 22ms/step
Prediction for myImages/three2.png: peace
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/rock.png: rock
1/1 [==============================] - 0s 22ms/step
Prediction for myImages/ok.jpg: ok
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/two_up_inverted.png: two_up_inverted
